In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import oracledb
import random

from flask import *
from flask_login import *
from langchain_huggingface import HuggingFaceEmbeddings
import psycopg2
from psycopg2 import pool

In [ ]:
def userdata2dict(d, j=[], l=[], f=[], fe=[]):
    return {
        'ID': d[0],
        'PW': d[1],
        'NAME': d[2],
        'NICK': d[3],
        'BIRTH': d[4].now().strftime('%Y-%m-%d'),
        'IMG_ID': d[5],
        'JOINED_AT': d[6].now().strftime('%Y-%m-%d'),
        'TYPE': d[7],
        'JOIN_CH': j,
        'LIKE_CH': l,
        'FOLLOW': f,
        'FOLLOWER': fe
    }

def chaldata2dict(d):
    return {
        'ID': d[0],
        'TITLE': d[1],
        'DETAIL': d[2].read(),
        'CREATED_AT': d[3].now().strftime('%Y-%m-%d'),
        'ADMIN': d[4],
        'IMG_ID': d[5],
        'JOIN_LIST': d[6],
        'LIKE_LIST': d[7],
        'POST_LIST': d[8]
    }

def postdata2dict(d, l=[], c=[]):
    return {
        'ID': d[0],
        'TITLE': d[1],
        'CONTENT': d[2].read(),
        'POSTED_AT': d[3].strftime('%Y-%m-%d %H:%M:%S'),
        'VIEWS': d[4],
        'WRITER': d[5],
        'CHAL_IDX': d[6],
        'IMG_ID': d[7],
        'LIKE_LIST': l,
        'CMT_LIST': c
    }

def cmtdata2dict(d):
    return {
        'ID': d[0],
        'POST_IDX': d[1],
        'CONTENT': d[2],
        'COMMENTED_AT': d[3].now().strftime('%Y-%m-%d'),
        'WRITER': d[4],
        'LIKE_LIST': d[5]
    }

In [123]:
#region MODEL

class User(UserMixin):
    def __init__(self, info):
        self.info = info

    #region getter

    def get_id(self):
        return self.info.get('ID')
    def get_name(self):
        return self.info.get('NAME')
    def get_birth(self):
        return self.info.get('BIRTH')
    def get_img_id(self):
        return self.info.get('IMG_ID')
    def get_joinCh(self):
        return self.info.get('JOIN_CH')
    def get_likeCh(self):
        return self.info.get('LIKE_CH')
    def get_follow(self):
        return self.info.get('FOLLOW')
    #endregion
    
    @staticmethod
    def get_user_info(userEmail):
        with con.cursor() as cur:
            userData = cur.execute("select * from members where mb_email=:1", (userEmail,)).fetchone()
            if userData == None:
                return None
            userData = list(userData)
            data = cur.execute("SELECT CHAL_IDX FROM JOIN_CHALLENGES WHERE MB_EMAIL=:1", (userEmail,)).fetchall()
            j = [d[0] for d in data] if data != None else []
            data = cur.execute("SELECT CHAL_IDX FROM LIKE_CHALLENGES WHERE MB_EMAIL=:1", (userEmail,)).fetchall()
            l = [d[0] for d in data] if data != None else []
            data = cur.execute(
                "SELECT FOLLOW FROM FOLLOWS WHERE FOLLOWER=:1",
                (userEmail,)
            )
            f = [d[0] for d in data] if data != None else []
            data = cur.execute(
                "SELECT FOLLOWER FROM FOLLOWS WHERE FOLLOW=:1",
                (userEmail,)
            )
            fe = [d[0] for d in data] if data != None else []
        return userdata2dict(userData, j=j, l=l, f=f, fe=fe)

    @staticmethod
    def get_user_info_simple(userEmail):
        with con.cursor() as cur:
            userData = cur.execute("select * from members where mb_email=:1", (userEmail,)).fetchone()
            if userData == None:
                return None
        return userdata2dict(userData)
    
class Challenge():
    def __init__(self, info):
        self.info = info
        
    #region getter

    def get_id(self):
        return self.info.get('ID')
    def get_title(self):
        return self.info.get('TITLE')
    def get_detail(self):
        return self.info.get('DETAIL')
    def get_created_at(self):
        return self.info.get('CREATED_AT')
    def get_admin(self):
        return self.info.get('ADMIN')
    def get_img(self):
        return self.info.get('IMG_ID')
    def get_join_list(self):
        return self.info.get('JOIN_LIST')
    def get_like_list(self):
        return self.info.get('LIKE_LIST')
    def get_post_list(self):
        return self.info_get('POST_LIST')

    #endregion
    
    @staticmethod
    def get_challenge_info(chal_idx):
        with con.cursor() as cur:
            chalData = cur.execute("SELECT * FROM CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchone()
            if chalData == None:
                return None
            chalData = list(chalData)
            data = cur.execute("SELECT MB_EMAIL FROM JOIN_CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchall()
            chalData.append([d[0] for d in data] if data != None else [])
            data = cur.execute("SELECT MB_EMAIL FROM LIKE_CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchall()
            chalData.append([d[0] for d in data] if data != None else [])
            data = cur.execute("SELECT POST_IDX FROM POSTS WHERE CHAL_IDX=:1 ORDER BY POSTED_AT", (chal_idx,)).fetchall()
            chalData.append([d[0] for d in data] if data != None else [])
        return chaldata2dict(chalData)

    @staticmethod
    def get_challenge_info_simple(chal_idx):
        with con.cursor() as cur:
            d = cur.execute("SELECT CHAL_TITLE, CHAL_INFO, THUMBNAIL_IMG FROM CHALLENGES WHERE CHAL_IDX=:1", (chal_idx,)).fetchone()
            if d == None:
                return None
        return {
            'ID': chal_idx,
            'TITLE': d[0],
            'DETAIL': d[1].read(),
            'IMG_ID': d[2]
        }

class Post():
    def __init__(self, info):
        self.info = info
    
    #region getter

    def get_id(self):
        return self.info.get('ID')
    def get_title(self):
        return self.info.get('TITLE')
    def get_content(self):
        return self.info.get('DETAIL')
    def get_posted_at(self):
        return self.info.get('POSTED_AT')
    def get_views(self):
        return self.info.get('VIEWS')
    def get_writer(self):
        return self.info.get('WRITER')
    def get_chal_idx(self):
        return self.info.get('CHAL_IDX')
    def get_img_id(self):
        return self.info.get('IMG_ID')
    def get_like_list(self):
        return self.info.get('LIKE_LIST')
    def get_cmt_list(self):
        return self.info.get('CMT_LIST')

    #endregion

    @staticmethod
    def get_post_info(post_idx):
        with con.cursor() as cur:
            postData = cur.execute("SELECT * FROM POSTS WHERE POST_IDX=:1",(post_idx,)).fetchone()
            if postData == None:
                return None
            postData = list(postData)
            data = cur.execute("SELECT MB_EMAIL FROM LIKE_POSTS WHERE POST_IDX=:1", (post_idx,)).fetchall()
            l = ([d[0] for d in data] if data != None else [])
            data = cur.execute(
                "SELECT CMT_IDX FROM COMMENTS WHERE POST_IDX=:1 ORDER BY COMMENTED_AT",
                (post_idx,)
            ).fetchall()
            c = ([d[0] for d in data] if data != None else [])
        return postdata2dict(postData, l, c)

    @staticmethod
    def get_post_info_simple(post_idx):
        with con.cursor() as cur:
            d = cur.execute("SELECT * FROM POSTS WHERE POST_IDX=:1",(post_idx,)).fetchone()
            if d == None:
                return None
            like = cur.execute("SELECT count(MB_EMAIL) FROM LIKE_POSTS WHERE POST_IDX=:1", (post_idx,)).fetchone()[0]
        return {
            'ID': post_idx,
            'TITLE': d[1],
            'VIEWS': d[4],
            'WRITER': d[5],
            'LIKE': like
        }

class Comment():
    def __init__(self, info):
        self.info = INFO
    
    #region getter

    #endregion

    @staticmethod
    def get_comment_info(cmt_idx):
        with con.cursor() as cur:
            cmtData = cur.execute("SELECT * FROM COMMENTS WHERE CMT_IDX=:1", (cmt_idx,)).fetchone()
            if cmtData == None:
                return None
            cmtData = list(cmtData)
            data = cur.execute("SELECT MB_EMAIL FROM LIKE_COMMENTS WHERE CMT_IDX=:1", (cmt_idx,)).fetchall()
            cmtData.append([d[0] for d in data] if data != None else [])
        return cmtdata2dict(cmtData)
    
#endregion

#region INIT

app = Flask(__name__)
app.secret_key = 'qalgmwjnfcndlemfyporpsltems'

lm = LoginManager()
lm.init_app(app)

@lm.user_loader
def user_loader(userId):
    userInfo = User.get_user_info(userId)
    return User(userInfo)

@lm.unauthorized_handler
def unauthorized():
    return redirect('/')

HFmodel = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

SQLALCHEMY_ENGINE_OPTIONS = {"pool_pre_ping": True}
SQLALCHEMY_TRACK_MODIFICATIONS = False

#endregion

#region VIEW

@app.route('/loginform')
def loginform():
    if current_user.is_authenticated:
        return redirect('/')
    else:
        return render_template('loginform.html')

@app.route('/joinform')
def joinform():
    return render_template('joinform.html')

@app.route('/challenge', methods = ['get', 'post'])
def challenge():
    index = request.args.get('index', 1)
    chalData = Challenge.get_challenge_info(index)
    if chalData == None:
        return redirect('/')
    adminName = User.get_user_info_simple(chalData['ADMIN'])['NICK']
    postInfo = {d: Post.get_post_info(d) for d in chalData['POST_LIST']}
    userInfo = {postInfo[i]['WRITER']: User.get_user_info_simple(postInfo[i]['WRITER']) for i in postInfo}
    return render_template(
        'challenge.html',
        adminName = adminName,
        chInfo = chalData,
        postInfo = postInfo,
        userInfo = userInfo
    )

@app.route('/challenge/postform', methods = ['get', 'post'])
@login_required
def postform():
    index = request.args.get('index', 1)
    chInfo = Challenge.get_challenge_info(index)
    if chInfo == None:
        return redirect('/')
    return render_template(
        'newpostform.html',
        chInfo = chInfo
    )

@app.route('/post', methods = ['get', 'post'])
def post():
    index = request.args.get('index', 1)
    postData = Post.get_post_info(index)
    if postData == None:
        return redirect('/')
    cmtInfo = {d: Comment.get_comment_info(d) for d in postData['CMT_LIST']}
    userList = [cmtInfo[i]['WRITER'] for i in cmtInfo] +[postData['WRITER']]
    userInfo = {i: User.get_user_info_simple(i) for i in userList}
    return render_template(
        'post.html',
        postInfo = postData,
        cmtInfo = cmtInfo,
        userInfo = userInfo
    )

@app.route('/user/<userId>')
def userinfo(userId):
    userData = User.get_user_info(userId)
    if userData == None:
        return "ERROR"
    with con.cursor() as cur:
        data = cur.execute(
            "SELECT POST_IDX FROM POSTS WHERE MB_EMAIL=:1",
            (userId,)
        ).fetchall()
        postInfo = {d[0]: Post.get_post_info(d[0]) for d in data} if data != None else {}
    chalList = []
    chalList += userData['JOIN_CH']
    chalList += userData['LIKE_CH']
    chalList = set(chalList)
    return render_template(
        'userpage.html',
        userInfo = userData,
        postInfo = postInfo,
        chalInfo = {i: Challenge.get_challenge_info(i) for i in chalList}
    )
    
@app.route('/userhome')
@login_required
def userhome():
    userId = current_user.get_id()
    userData = User.get_user_info(userId)
    if userData == None:
        return "ERROR"
    with con.cursor() as cur:
        data = cur.execute(
            "SELECT * FROM POSTS WHERE MB_EMAIL IN (SELECT FOLLOW FROM FOLLOWS WHERE FOLLOWER=:1) ORDER BY POSTED_AT",
            (userData['ID'],)
        ).fetchall()
        postInfo = {d[0]: postdata2dict(d) for d in data}
    return render_template(
        'userhome.html',
        postInfo = postInfo
    )

@app.route('/newchallengeform') 
@login_required
def newchallengeform():
    return render_template('newchallengeform.html')

@app.route('/')
def index():
    chalList = []
    postInfo = {}
    if current_user.is_authenticated:
        userId = current_user.get_id()
        chalList += current_user.get_joinCh()
        chalList += current_user.get_likeCh()
        with pool.getconn().cursor() as pcur:
            pcur.execute(
                f"select id, embedding<=>(select embedding from uservector where id='{userId}') from challengeembeddings"
            )
            res = pcur.fetchall()
        score = {r[0]: 1-r[1] for r in res}
    else:
        with con.cursor() as cur:
            res = cur.execute(
                "select chal_idx from challenges"
            ).fetchall()
        score = {r[0]: 0 for r in res}
    with con.cursor() as cur:
        likeRes = cur.execute("select chal_idx, count(chal_idx) from like_challenges group by chal_idx").fetchall()
        joinRes = cur.execute("select chal_idx, count(chal_idx) from join_challenges group by chal_idx").fetchall()
    if current_user.is_authenticated:
        with con.cursor() as cur:
            userJoinRes = cur.execute(
                "select chal_idx from join_challenges where MB_EMAIL=:1",
                (userId,)
            ).fetchall()
        userJoinRes = [r[0] for r in userJoinRes]
    else:
        userJoinRes = []
    likeRes = {r[0]: r[1] for r in likeRes}
    joinRes = {r[0]: r[1] for r in joinRes}
    for i in score:
        score[i] *= (likeRes[i]*0.1 +joinRes[i])
    for i in userJoinRes:
        score[i] = 0
    score = sorted(score, key=lambda x: -score[x])
    chalList += score

    chalList = set(chalList)
    return render_template(
        'mainpage.html',
        recList = score,
        chalInfo = {i: Challenge.get_challenge_info(i) for i in chalList},
        postInfo = postInfo
    )

@app.route('/test')
def test():
    return render_template('testPage.html')
    
#endregion

#region CONTROLLER

@app.route('/login', methods = ['get', 'post'])
def login():
    params = request.get_json()
    userId = params['userId']
    userPw = params['userPw']

    if  userId == '' or userPw == '':
        return jsonify({"result": 0, "msg": "아이디와 비밀번호를 입력해주세요."})
    
    userData = User.get_user_info(userId)
    if userData is None or userData['PW'] != userPw:
        return jsonify({"result": 0, "msg": "아이디 또는 비밀번호가 일치하지 않습니다."})
    
    loginUser = User(userData)
    login_user(loginUser)
    return jsonify({"result":1, "msg": url_for('index')})

@app.route('/logout')
def logout():
    logout_user()
    return redirect('/')

@app.route('/joinEmail', methods = ['get', 'post'])
def joinEmail():
    params = request.get_json()
    if params['id'] == '' or params['domain'] == '':
        return jsonify({"result": 0, "msg": "이메일을 입력해주세요."})
    
    userData = User.get_user_info(params['id'] +'@' +params['domain'])
    # print(userData)
    if userData is not None:
        return jsonify({"result": 0, "msg": "이미 사용 중인 이메일입니다."})
    else:
        return jsonify({"result": 1, "msg": "사용 가능한 이메일입니다."})

@app.route('/join', methods = ['get', 'post'])
def join():
    params = request.get_json()
    userId = params['ID']
    userPw = params['PW']
    userName = params['NAME']
    userNick = params['NICK']
    userBirth = params['BIRTH']

    emailRegulation = re.compile('^[\w\-\.]+\@[\w\-]+\.[\w\-]{2,4}$')
    pwRegulation = re.compile('^(?=.*[0-9])(?=.*[a-zA-Z])(?=.*[\!\@\#\$\^\&\*\(\)])[0-9a-zA-Z\!\@\#\$\^\&\*\(\)]{8,16}$')

    msg = {
        'result': 1,
        'email': 0,
        'pw': 0,
        'name': 0,
        'nick': 0,
        'birth': 0
    }

    if User.get_user_info(userId):
        msg['result'] = 0
        msg['email'] = '이미 사용 중인 이메일입니다.'
    if emailRegulation.match(userId) is None:
        msg['result'] = 0
        msg['email'] = '유효하지 않은 이메일입니다.'

    if pwRegulation.match(userPw) is None:
        msg['result'] = 0
        msg['pw'] = '유효하지 않은 비밀번호입니다.'

    if userName == '':
        msg['result'] = 0
        msg['name'] = '이름은 비워둘 수 없습니다.'

    if userNick == '':
        msg['result'] = 0
        msg['nick'] = '이름은 비워둘 수 없습니다.'

    if userBirth == '--':
        msg['result'] = 0
        msg['birth'] = '생일은 비워둘 수 없습니다.'

    if msg['result'] == 0:
        return jsonify(msg)

    with con.cursor() as cur:
        cur.execute("INSERT INTO MEMBERS VALUES(:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'), NULL, SYSDATE, 'USER')", (userId, userPw, userName, userNick, userBirth))
        con.commit()

    return jsonify({'result': 1, 'msg':'가입 성공'})

@app.route('/challengeinfo', methods = ['get', 'post'])
def challengeinfo():
    index = request.args.get('index', 1)
    return jsonify(Challenge.get_challenge_info(index))

@app.route('/challengeinfo/likeNum', methods = ['get', 'post'])
def challengeinfo_likeNum():
    index = request.args.get('index', 1)
    with con.cursor() as cur:
        data = cur.execute("SELECT MB_EMAIL FROM LIKE_CHALLENGES WHERE CHAL_IDX=:1", (index,)).fetchall()
    return jsonify({'likeNum': len(data)})

@app.route('/challengeinfo/joinList', methods = ['get', 'post'])
def challengeinfo_joinList():
    index = request.args.get('index', 1)
    with con.cursor() as cur:
        data = cur.execute("SELECT MB_EMAIL FROM JOIN_CHALLENGES WHERE CHAL_IDX=:1", (index,)).fetchall()
    return jsonify({'joinList': [d[0] for d in data]})

@app.route('/newchallenge', methods = ['get', 'post'])
@login_required
def newchallenge():
    params = request.get_json()
    title = params['TITLE']
    detail = params['DETAIL']
    img_id = params['IMG_ID']
    if title == '' or detail == '':
        return jsonify({"result": 0})
    with con.cursor() as cur:
        cur.execute(
            "INSERT INTO CHALLENGES VALUES(0, :1, :2, SYSDATE, :3, :4)",
            (title, detail, current_user.get_id(), img_id)
        )
        con.commit()
        return jsonify({"result": 1})

@app.route('/checkch', methods = ['get', 'post'])
@login_required
def challengeInter():
    index = int(request.args.get('index', 1))
    check = int(request.args.get('check', 1))
    typ = request.args.get('type', '')
    userId = current_user.get_id()
    with con.cursor() as cur:
        if typ == '1':
            if index in current_user.get_likeCh():
                cur.execute("DELETE FROM LIKE_CHALLENGES WHERE MB_EMAIL=:1 AND CHAL_IDX=:2", (userId, index))
                con.commit()
                return jsonify({'result': 1})
            else:
                cur.execute("INSERT INTO LIKE_CHALLENGES VALUES(:1, :2)", (userId, index))
                con.commit()
                return jsonify({'result': 1})
        elif typ == '2':
            if index in current_user.get_joinCh():
                cur.execute("DELETE FROM JOIN_CHALLENGES WHERE MB_EMAIL=:1 AND CHAL_IDX=:2", (userId, index))
                con.commit()
                return jsonify({'result': 1})
            else:
                cur.execute("INSERT INTO JOIN_CHALLENGES VALUES(:1, :2)", (userId, index))
                con.commit()
                return jsonify({'result': 1})
    return jsonify({'result': 0})

@app.route('/follow', methods = ['get', 'post'])
@login_required
def follow():
    userId = request.args.get('id', '')
    nowId = current_user.get_id()
    if userId == '':
        return jsonify({'result': 0})
    follower = User.get_user_info(userId)['FOLLOWER']

    with con.cursor() as cur:
        if nowId in follower:
            cur.execute(
                "DELETE FROM FOLLOWS WHERE FOLLOW=:1 AND FOLLOWER=:2",
                (userId, nowId)
            )
        else:
            cur.execute(
                "INSERT INTO FOLLOWS VALUES(:1, :2)",
                (userId, nowId)
            )
        con.commit()
    return jsonify({'result': 1})

@app.route('/challengeProgress', methods = ['get', 'post'])
@login_required
def challengeProgress():
    index = request.args.get('index', 1)
    
    return jsonify({
        "progress": [random.randint(0, 100) for _ in range(100)]
    })

@app.route('/postinfo', methods = ['get', 'post'])
def postinfo():
    index = request.args.get('index', 1)
    with con.cursor() as cur:
        data = cur.execute(
            "SELECT MB_EMAIL FROM LIKE_POSTS WHERE POST_IDX=:1",
            (index,)
        ).fetchall()
        l = ([d[0] for d in data] if data != None else [])
        data = cur.execute(
            "SELECT CMT_IDX FROM COMMENTS WHERE POST_IDX=:1 ORDER BY COMMENTED_AT",
            (index,)
        ).fetchall()
        c = ([d[0] for d in data] if data != None else [])
    return jsonify({'like': l, 'cmt': c})

@app.route('/newpost', methods = ['get', 'post'])
@login_required
def newPost():
    params = request.get_json()
    content = params['CONTENT']
    imgId = params['IMG_ID']
    index = params['INDEX']
    embedding = HFmodel.embed_query(content)

    if content == '':
        return jsonify({"result": 0})

    with con.cursor() as cur:
        cur.execute(
            "INSERT INTO POSTS VALUES(0, :1, :2, SYSDATE, 0, :3, :4, :5)",
            ("TITLE", content, current_user.get_id(), index, imgId)
        )
        postId = cur.execute("SELECT MAX(POST_IDX) FROM POSTS").fetchone()[0]
    con.commit()

    # with pool.getconn() as pcon:
    #     with pcon.cursor() as pcur:
    #         pcur.execute(
    #             f"INSERT INTO POSTEMBEDDINGS VALUES('{postId}', '{embedding}'::vector)"
    #         )
    #         pcur.execute(
    #             f"update uservector set embedding = uservector.embedding+'{embedding}'::vector where id='{current_user.get_id()}'"
    #         )
    #         pcur.execute(
    #             f"update challengeembeddings set embedding = challengeembeddings.embedding+'{embedding}'::vector where id={index}"
    #         )
    #     pcon.commit()

    return jsonify({"result": 1})
    
@app.route('/commentinfo', methods = ['get', 'poset'])
def commentinfo():
    index = request.args.get('index', 1)
    return jsonify(Comment.get_comment_info(index))

@app.route('/newcomment', methods = ['get', 'post'])
@login_required
def newComment():
    params = request.get_json()
    content = params['CONTENT']
    index = params['INDEX']
    if content == '':
        return jsonify({"result": 0})
    with con.cursor() as cur:
        cur.execute(
            "INSERT INTO COMMENTS VALUES(0, :1, :2, SYSDATE, :3)",
            (index, content, current_user.get_id())
        )
        con.commit()
        return jsonify({"result": 1})

@app.route('/nextpost', methods = ['get', 'post'])
@login_required
def nextfeed():
    userId = current_user.get_id()
    mn = int(request.args.get('feednum', -1))+1
    if mn == 0:
        return jsonify({'result': 0})
    mx = mn +19
    with con.cursor() as cur:
            data = cur.execute(
            f"select * from (SELECT row_number() over (order by posted_at) num, POSTS.* FROM POSTS WHERE MB_EMAIL IN (SELECT FOLLOW FROM FOLLOWS WHERE FOLLOWER=:1) OR MB_EMAIL=:1 ORDER BY POSTED_AT) where num between {mn} and {mx}",
            (userId,)
        ).fetchall()
    postInfo = {str(d[1]): postdata2dict(d[1:]) for d in data}
    userInfo = {postInfo[i]['WRITER']: User.get_user_info_simple(postInfo[i]['WRITER']) for i in postInfo}
    key = [d[1] for d in data]
    print(userInfo)
    return jsonify({'key':key, 'postInfo': postInfo, 'userInfo': userInfo})

@app.route('/usernextpost', methods = ['get', 'post'])
def usernextfeed():
    userId = request.args.get('user', '')
    mn = int(request.args.get('feednum', -1))+1
    if mn == 0 or userId == '':
        return jsonify({'result': 0})
    mx = mn +19
    with con.cursor() as cur:
            data = cur.execute(
            f"select * from (SELECT row_number() over (order by posted_at) num, POSTS.* FROM POSTS WHERE MB_EMAIL=:1 ORDER BY POSTED_AT DESC) where num between {mn} and {mx}",
            (userId,)
        ).fetchall()
    postInfo = {str(d[1]): postdata2dict(d[1:]) for d in data}
    userInfo = {userId: User.get_user_info_simple(userId)}
    key = [d[1] for d in data]
    print(userInfo)
    return jsonify({'key':key, 'postInfo': postInfo, 'userInfo': userInfo})
    
# endregion

In [124]:
oracledb.init_oracle_client()

with open('apikey.json', mode='r') as f:
    key = json.load(f)

pool = psycopg2.pool.SimpleConnectionPool(1, 20,
    host=key['gres_host'],
    dbname=key['gres_dbname'],
    user=key['gres_user'],
    password=key['gres_password'],
    port=key['gres_port'])
with oracledb.connect(
    user=key['oracle_user'],
    password=key['oracle_password'],
    dsn=key['oracle_dsn']) as con:
    app.run(host='127.0.0.1', port=5050)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5050
Press CTRL+C to quit
127.0.0.1 - - [03/Jul/2024 11:54:52] "GET /challenge?index=60 HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2024 11:54:52] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Jul/2024 11:54:52] "GET /static/css/challenge.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Jul/2024 11:54:52] "GET /static/img/Logo.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Jul/2024 11:54:52] "GET /static/js/challenge.js HTTP/1.1" 304 -
